In [1]:
import pandas as pd
import numpy as np
import scipy
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.ensemble import IsolationForest, AdaBoostClassifier, BaggingClassifier, BaseEnsemble, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
import eli5
import shap
from sklearn import model_selection
from imblearn.under_sampling import TomekLinks
import umap

pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [31]:
train = pd.read_csv('../data/CAX_MortgageModeling_Train.csv')

In [32]:
train.RESULT = train.RESULT.apply(lambda x: 1 if x == 'FUNDED' else 0)
train.RESULT = train.RESULT.apply(lambda x: 1-x)

In [33]:
test = pd.read_csv('../data/CAX_MortgageModeling_Test.csv')

In [34]:
train_len = train.shape[0]
traincols = list(train.columns[2:-1])

In [35]:
data = pd.concat([train[['Unique_ID'] + traincols+['RESULT']],test[['Unique_ID']+traincols+['RESULT']]],axis=0)

In [36]:
data['GDS'] = np.abs(data['GDS'])
data['GDS'] = np.clip(data.GDS,0,data['GDS'].max())

In [37]:
data['TDS'] = np.abs(data['TDS'])
data['TDS'] = np.clip(data.TDS,0,data['TDS'].max())

In [38]:
data['GDS'] = data['GDS']/100
data['LTV'] = data['LTV']/100
data['TDS'] = data['TDS']/100
data['RATE'] = data['RATE']/100

In [39]:
data['APPRAISED PROPERTY VALUE'] = data['MORTGAGE AMOUNT']/data['LTV']
data['CHANGE IN PROPERTY VALUE'] = (data['APPRAISED PROPERTY VALUE']-data['PROPERTY VALUE'])/data['PROPERTY VALUE']
data['ANNUALIZED HOUSING EXPENSE'] = data['GDS'] * data['INCOME']
data['MORTGAGE RATIO'] = data['MORTGAGE AMOUNT']/data['PROPERTY VALUE']
data['OTHER EXPENSE'] = data['INCOME'] * data['TDS'] - data['ANNUALIZED HOUSING EXPENSE']
data['RENT INCOME'] = data.apply(lambda x: np.abs(x['OTHER EXPENSE']) if x.GDS > x.TDS else 0, axis=1)
data['OTHER EXPENSE'] = np.clip(data['OTHER EXPENSE'],0,data['OTHER EXPENSE'].max())

In [40]:
def f(x):
    if x['PAYMENT FREQUENCY'] in ['Bi-Weekly','Bi-Weekly Accelerated','Semi-Monthly'] :
        return 2
    elif x['PAYMENT FREQUENCY'] in ['Weekly','Weekly Accelerated']:
        return 4
    else:
        return x['PAYMENT FREQUENCY IN MONTHS']
data['PAYMENT FREQUENCY IN MONTHS'] = 1
data['PAYMENT FREQUENCY IN MONTHS'] = data.apply(lambda x: f(x), axis=1)

In [41]:
data['MORTGAGE PAYMENT'] = data['PAYMENT FREQUENCY IN MONTHS'] * data['MORTGAGE PAYMENT']

In [42]:
data['ACTUAL MORTGAGE PAYMENT'] = data.apply(lambda x: x['MORTGAGE AMOUNT'] * np.exp(x.RATE*x.AMORTIZATION*1.0/12), axis=1)
data['TOTAL INTEREST'] = data.apply(lambda x: x['ACTUAL MORTGAGE PAYMENT'] - x['MORTGAGE AMOUNT'], axis=1)

In [43]:
data['MORTGAGE AMOUNT PER INCOME'] = data['MORTGAGE AMOUNT']/data['INCOME']
data['MORTGAGE PAYMENT PER INCOME'] = data['MORTGAGE PAYMENT']*12/data['INCOME']
data['PROPERTY VALUE PER INCOME'] = data['PROPERTY VALUE']/data['INCOME']
data['TOTAL INTEREST PER INCOME'] = data['TOTAL INTEREST']/data['INCOME']
data['OTHER EXPENSE PER INCOME'] = data['OTHER EXPENSE']/data['INCOME']
data['ANNUALIZED HOUSING EXPENSE PER INCOME'] = data['ANNUALIZED HOUSING EXPENSE']/data['INCOME']

In [44]:
data['ACTUAL MORTGAGE PAYMENT BY MORTGAGE'] = data['ACTUAL MORTGAGE PAYMENT']/data['MORTGAGE AMOUNT']

In [45]:
data['FSA'] = data['FSA'].apply(lambda x: str(x[:2]))

In [46]:
data.replace('Under 25',25,inplace=True)
data.replace('25-29',29,inplace=True)
data.replace('30-34',34,inplace=True)
data.replace('35-39',39,inplace=True)
data.replace('40-44',44,inplace=True)
data.replace('45-49',49,inplace=True)
data.replace('50-54',54,inplace=True)
data.replace('55-59',59,inplace=True)
data.replace('60-64',64,inplace=True)
data.replace('65-69',69,inplace=True)
data.replace('70 and over',75,inplace=True)

In [47]:
data.shape

(60856, 37)

In [48]:
print (data[['PROPERTY VALUE','INCOME','FSA','CREDIT SCORE']].drop_duplicates().shape)

(60525, 4)


In [49]:
cnt_cols = []
cat_cols = []
for col in data.columns:
    if data[col].nunique() > 50 and data[col].dtype != object:
        cnt_cols.append(col)
    elif col != 'RESULT':
        cat_cols.append(col)

In [50]:
cat_cols

['Unique_ID',
 'AMORTIZATION',
 'RATE',
 'MORTGAGE PURPOSE',
 'PAYMENT FREQUENCY',
 'PROPERTY TYPE',
 'TERM',
 'FSA',
 'AGE RANGE',
 'GENDER',
 'INCOME TYPE',
 'NAICS CODE',
 'RENT INCOME',
 'PAYMENT FREQUENCY IN MONTHS']

In [51]:
cnt_cols

['PROPERTY VALUE',
 'MORTGAGE PAYMENT',
 'GDS',
 'LTV',
 'TDS',
 'MORTGAGE AMOUNT',
 'INCOME',
 'CREDIT SCORE',
 'APPRAISED PROPERTY VALUE',
 'CHANGE IN PROPERTY VALUE',
 'ANNUALIZED HOUSING EXPENSE',
 'MORTGAGE RATIO',
 'OTHER EXPENSE',
 'ACTUAL MORTGAGE PAYMENT',
 'TOTAL INTEREST',
 'MORTGAGE AMOUNT PER INCOME',
 'MORTGAGE PAYMENT PER INCOME',
 'PROPERTY VALUE PER INCOME',
 'TOTAL INTEREST PER INCOME',
 'OTHER EXPENSE PER INCOME',
 'ANNUALIZED HOUSING EXPENSE PER INCOME',
 'ACTUAL MORTGAGE PAYMENT BY MORTGAGE']

In [52]:
data['Is unemployed'] = data['AGE RANGE'].apply(lambda x: 1 if x == 25 else 0)
data['Is recently married'] = data['AGE RANGE'].apply(lambda x: 1 if x > 25 and x <= 34 else 0)
data['Is married'] = data['AGE RANGE'].apply(lambda x: 1 if x > 34 else 0)
data['Is established'] = data['AGE RANGE'].apply(lambda x: 1 if x >= 39 else 0)
data['Is about to retire'] = data['AGE RANGE'].apply(lambda x: 1 if x >= 55 else 0)
data['Is retired'] = data['AGE RANGE'].apply(lambda x: 1 if x >= 64 else 0)

In [53]:
data['REPAYMENT AGE'] = data.AMORTIZATION/12 + data['AGE RANGE']

In [54]:
for cat_col in ['MORTGAGE PURPOSE','PAYMENT FREQUENCY','PROPERTY TYPE','AGE RANGE','GENDER','INCOME TYPE','FSA']:
    for cnt_col in cnt_cols:
        temp = data.groupby([cat_col])[cnt_col].agg(['mean','var']).reset_index()
        temp.columns = [cat_col,cnt_col+' ' + cat_col+' MEAN',cnt_col+' ' + cat_col+' VAR']
        data = pd.merge(data,temp,how='left')
        data[cnt_col+' ' + cat_col+' Z SCORE'] = data.apply(lambda x: (x[cnt_col]-x[cnt_col+' ' + cat_col+' MEAN'])/np.sqrt(x[cnt_col+' ' + cat_col+' VAR']), axis=1)
        data = data.drop([cnt_col+' ' + cat_col+' MEAN',cnt_col+' ' + cat_col+' VAR'],axis=1)

In [55]:
data = pd.get_dummies(data,columns=['MORTGAGE PURPOSE','PAYMENT FREQUENCY','PROPERTY TYPE','FSA','GENDER','AMORTIZATION','TERM','INCOME TYPE'])

In [56]:
data = data.drop(['Unique_ID','PAYMENT FREQUENCY IN MONTHS','NAICS CODE'], axis=1)

In [57]:
traincols = list(data.columns)
traincols.remove('RESULT')

In [58]:
binary_cols = []
for col in traincols:
    if data[col].nunique() == 2:
        binary_cols.append(col)

In [59]:
#embedding = umap.UMAP().fit_transform(data[binary_cols])
data['umap_1'] = embedding[:,0]
data['umap_2'] = embedding[:,1]

In [60]:
data.replace(np.nan,0,inplace=True)
data.replace(np.inf,0,inplace=True)

In [61]:
data.to_csv('../data/data_full_final_1705.csv',index=False)

In [77]:
data = pd.read_csv('../data/data_full_final.csv')

In [62]:
data.head(10)

,PROPERTY VALUE,MORTGAGE PAYMENT,GDS,LTV,TDS,MORTGAGE AMOUNT,RATE,AGE RANGE,INCOME,CREDIT SCORE,RESULT,APPRAISED PROPERTY VALUE,CHANGE IN PROPERTY VALUE,ANNUALIZED HOUSING EXPENSE,MORTGAGE RATIO,OTHER EXPENSE,RENT INCOME,ACTUAL MORTGAGE PAYMENT,TOTAL INTEREST,MORTGAGE AMOUNT PER INCOME,MORTGAGE PAYMENT PER INCOME,PROPERTY VALUE PER INCOME,TOTAL INTEREST PER INCOME,OTHER EXPENSE PER INCOME,ANNUALIZED HOUSING EXPENSE PER INCOME,ACTUAL MORTGAGE PAYMENT BY MORTGAGE,Is unemployed,Is recently married,Is married,Is established,Is about to retire,Is retired,REPAYMENT AGE,PROPERTY VALUE MORTGAGE PURPOSE Z SCORE,MORTGAGE PAYMENT MORTGAGE PURPOSE Z SCORE,GDS MORTGAGE PURPOSE Z SCORE,LTV MORTGAGE PURPOSE Z SCORE,TDS MORTGAGE PURPOSE Z SCORE,MORTGAGE AMOUNT MORTGAGE PURPOSE Z SCORE,INCOME MORTGAGE PURPOSE Z SCORE,CREDIT SCORE MORTGAGE PURPOSE Z SCORE,APPRAISED PROPERTY VALUE MORTGAGE PURPOSE Z SCORE,CHANGE IN PROPERTY VALUE MORTGAGE PURPOSE Z SCORE,ANNUALIZED HOUSING EXPENSE MORTGAGE PURPOSE Z SCORE,MORTGAGE RATIO MORTGAGE PURPOSE Z SCORE,OTHER EXPENSE MORTGAGE PURPOSE Z SCORE,ACTUAL MORTGAGE PAYMENT MORTGAGE PURPOSE Z SCORE,TOTAL INTEREST MORTGAGE PURPOSE Z SCORE,MORTGAGE AMOUNT PER INCOME MORTGAGE PURPOSE Z SCORE,MORTGAGE PAYMENT PER INCOME MORTGAGE PURPOSE Z SCORE,PROPERTY VALUE PER INCOME MORTGAGE PURPOSE Z SCORE,TOTAL INTEREST PER INCOME MORTGAGE PURPOSE Z SCORE,OTHER EXPENSE PER INCOME MORTGAGE PURPOSE Z SCORE,ANNUALIZED HOUSING EXPENSE PER INCOME MORTGAGE PURPOSE Z SCORE,ACTUAL MORTGAGE PAYMENT BY MORTGAGE MORTGAGE PURPOSE Z SCORE,PROPERTY VALUE PAYMENT FREQUENCY Z SCORE,MORTGAGE PAYMENT PAYMENT FREQUENCY Z SCORE,GDS PAYMENT FREQUENCY Z SCORE,LTV PAYMENT FREQUENCY Z SCORE,TDS PAYMENT FREQUENCY Z SCORE,MORTGAGE AMOUNT PAYMENT FREQUENCY Z SCORE,INCOME PAYMENT FREQUENCY Z SCORE,CREDIT SCORE PAYMENT FREQUENCY Z SCORE,APPRAISED PROPERTY VALUE PAYMENT FREQUENCY Z SCORE,CHANGE IN PROPERTY VALUE PAYMENT FREQUENCY Z SCORE,ANNUALIZED HOUSING EXPENSE PAYMENT FREQUENCY Z SCORE,MORTGAGE RATIO PAYMENT FREQUENCY Z SCORE,OTHER EXPENSE PAYMENT FREQUENCY Z SCORE,ACTUAL MORTGAGE PAYMENT PAYMENT FREQUENCY Z SCORE,TOTAL INTEREST PAYMENT FREQUENCY Z SCORE,MORTGAGE AMOUNT PER INCOME PAYMENT FREQUENCY Z SCORE,MORTGAGE PAYMENT PER INCOME PAYMENT FREQUENCY Z SCORE,PROPERTY VALUE PER INCOME PAYMENT FREQUENCY Z SCORE,TOTAL INTEREST PER INCOME PAYMENT FREQUENCY Z SCORE,OTHER EXPENSE PER INCOME PAYMENT FREQUENCY Z SCORE,ANNUALIZED HOUSING EXPENSE PER INCOME PAYMENT FREQUENCY Z SCORE,ACTUAL MORTGAGE PAYMENT BY MORTGAGE PAYMENT FREQUENCY Z SCORE,PROPERTY VALUE PROPERTY TYPE Z SCORE,MORTGAGE PAYMENT PROPERTY TYPE Z SCORE,GDS PROPERTY TYPE Z SCORE,LTV PROPERTY TYPE Z SCORE,TDS PROPERTY TYPE Z SCORE,MORTGAGE AMOUNT PROPERTY TYPE Z SCORE,INCOME PROPERTY TYPE Z SCORE,CREDIT SCORE PROPERTY TYPE Z SCORE,APPRAISED PROPERTY VALUE PROPERTY TYPE Z SCORE,CHANGE IN PROPERTY VALUE PROPERTY TYPE Z SCORE,ANNUALIZED HOUSING EXPENSE PROPERTY TYPE Z SCORE,MORTGAGE RATIO PROPERTY TYPE Z SCORE,OTHER EXPENSE PROPERTY TYPE Z SCORE,ACTUAL MORTGAGE PAYMENT PROPERTY TYPE Z SCORE,TOTAL INTEREST PROPERTY TYPE Z SCORE,MORTGAGE AMOUNT PER INCOME PROPERTY TYPE Z SCORE,MORTGAGE PAYMENT PER INCOME PROPERTY TYPE Z SCORE,PROPERTY VALUE PER INCOME PROPERTY TYPE Z SCORE,TOTAL INTEREST PER INCOME PROPERTY TYPE Z SCORE,OTHER EXPENSE PER INCOME PROPERTY TYPE Z SCORE,ANNUALIZED HOUSING EXPENSE PER INCOME PROPERTY TYPE Z SCORE,ACTUAL MORTGAGE PAYMENT BY MORTGAGE PROPERTY TYPE Z SCORE,PROPERTY VALUE AGE RANGE Z SCORE,MORTGAGE PAYMENT AGE RANGE Z SCORE,GDS AGE RANGE Z SCORE,LTV AGE RANGE Z SCORE,TDS AGE RANGE Z SCORE,MORTGAGE AMOUNT AGE RANGE Z SCORE,INCOME AGE RANGE Z SCORE,CREDIT SCORE AGE RANGE Z SCORE,APPRAISED PROPERTY VALUE AGE RANGE Z SCORE,CHANGE IN PROPERTY VALUE AGE RANGE Z SCORE,ANNUALIZED HOUSING EXPENSE AGE RANGE Z SCORE,MORTGAGE RATIO AGE RANGE Z SCORE,OTHER EXPENSE AGE RANGE Z SCORE,ACTUAL MORTGAGE PAYMENT AGE RANGE Z SCORE,TOTAL INTEREST AGE RANGE Z SCORE,MORTGAGE AMOUNT PER INCOME AGE RANGE

In [78]:
train = data.iloc[:train_len]
test = data.iloc[train_len:]

In [79]:
traincols = list(train.columns)
traincols.remove('RESULT')

In [80]:
print (len(traincols))

713


In [81]:
train2 = pd.concat([train[train.RESULT == 0].sample(frac=.3),train[train.RESULT == 1]],axis=0).sample(frac=1)

In [68]:
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(train):
    break

In [69]:
for train_index2, test_index2 in kf.split(train2):
    break

In [71]:
rf = RandomForestClassifier(n_estimators=1000)
rf.fit(train2[traincols].iloc[train_index2], train2.RESULT.iloc[train_index2])
pred = rf.predict(train2[traincols].iloc[test_index2])
print (accuracy_score(train2.RESULT.iloc[test_index2],pred))
print (confusion_matrix(train2.RESULT.iloc[test_index2],pred))
print (f1_score(train2.RESULT.iloc[test_index2],pred))

0.6551639745472344
[[1549  669]
 [ 740 1128]]
0.6155525238744884


In [72]:
gbm = GradientBoostingClassifier(max_depth=8,n_estimators=1000)
gbm.fit(train2[traincols].iloc[train_index2], train2.RESULT.iloc[train_index2])
pred = gbm.predict(train2[traincols].iloc[test_index2])
print (accuracy_score(train2.RESULT.iloc[test_index2],pred))
print (confusion_matrix(train2.RESULT.iloc[test_index2],pred))
print (f1_score(train2.RESULT.iloc[test_index2],pred))

0.6644640234948604
[[1584  634]
 [ 737 1131]]
0.6226259289843106


In [73]:
importances = gbm.feature_importances_
indices = importances.argsort()[::-1]
cols = traincols
count = 0
for i in range(len(cols)):
    count += 1
    if importances[indices[i]] > 0 and count < 50:
        print ("Column {} has importance {}".format(cols[indices[i]],importances[indices[i]]))

Column PAYMENT FREQUENCY_Monthly has importance 0.04153714293353343
Column CHANGE IN PROPERTY VALUE MORTGAGE PURPOSE Z SCORE has importance 0.028591708755899427
Column TOTAL INTEREST FSA Z SCORE has importance 0.025400252857281758
Column CHANGE IN PROPERTY VALUE PAYMENT FREQUENCY Z SCORE has importance 0.022421835531278703
Column umap_1 has importance 0.01706507535718056
Column TDS MORTGAGE PURPOSE Z SCORE has importance 0.016777248384419807
Column umap_2 has importance 0.016750919278299685
Column APPRAISED PROPERTY VALUE FSA Z SCORE has importance 0.013387466824094668
Column CREDIT SCORE INCOME TYPE Z SCORE has importance 0.012254142083782954
Column TDS FSA Z SCORE has importance 0.012237372973064846
Column MORTGAGE RATIO FSA Z SCORE has importance 0.011955920231004344
Column ACTUAL MORTGAGE PAYMENT FSA Z SCORE has importance 0.01167291984633375
Column CHANGE IN PROPERTY VALUE has importance 0.011378059964156344
Column TDS has importance 0.010886612588296575
Column CHANGE IN PROPERTY 

In [74]:
importances = rf.feature_importances_
indices = importances.argsort()[::-1]
cols = traincols
count = 0
for i in range(len(cols)):
    count += 1
    if importances[indices[i]] > 0 and count < 50:
        print ("Column {} has importance {}".format(cols[indices[i]],importances[indices[i]]))

Column CHANGE IN PROPERTY VALUE PAYMENT FREQUENCY Z SCORE has importance 0.010150019418283399
Column ACTUAL MORTGAGE PAYMENT FSA Z SCORE has importance 0.009866164450025926
Column TOTAL INTEREST FSA Z SCORE has importance 0.009689936086052951
Column APPRAISED PROPERTY VALUE FSA Z SCORE has importance 0.009639296170917553
Column CHANGE IN PROPERTY VALUE MORTGAGE PURPOSE Z SCORE has importance 0.009509047688697352
Column umap_2 has importance 0.009046347147320442
Column TDS MORTGAGE PURPOSE Z SCORE has importance 0.009042317952888604
Column ANNUALIZED HOUSING EXPENSE PAYMENT FREQUENCY Z SCORE has importance 0.008957846301750321
Column TDS PAYMENT FREQUENCY Z SCORE has importance 0.008936543793342907
Column CHANGE IN PROPERTY VALUE has importance 0.008829333890184464
Column PROPERTY VALUE FSA Z SCORE has importance 0.008211045417534065
Column OTHER EXPENSE PAYMENT FREQUENCY Z SCORE has importance 0.008186996462021071
Column OTHER EXPENSE PER INCOME PAYMENT FREQUENCY Z SCORE has importance

#### Hyper parameter tuning ####

In [75]:
from hyperopt import hp, tpe
from hyperopt.fmin import fmin

In [82]:
def objective(params):
    params = {'n_estimators': int(params['n_estimators']), 'max_depth': int(params['max_depth'])}
    clf = RandomForestClassifier(n_jobs=2,  **params, random_state=123)
    score = cross_val_score(clf, train2[traincols], train2.RESULT, scoring='f1', cv=KFold(5)).mean()
    print("F1 {:.3f} params {}".format(score, params))
    return score

space = {
    'n_estimators': hp.quniform('n_estimators', 25, 500, 25),
    'max_depth': hp.quniform('max_depth', 1, 10, 1)
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)

F1 0.566 params {'n_estimators': 75, 'max_depth': 2}
F1 0.598 params {'n_estimators': 300, 'max_depth': 3}
F1 0.594 params {'n_estimators': 200, 'max_depth': 4}
F1 0.600 params {'n_estimators': 200, 'max_depth': 6}
F1 0.610 params {'n_estimators': 300, 'max_depth': 9}
F1 0.599 params {'n_estimators': 300, 'max_depth': 6}
F1 0.562 params {'n_estimators': 275, 'max_depth': 2}
F1 0.596 params {'n_estimators': 150, 'max_depth': 4}
F1 0.613 params {'n_estimators': 325, 'max_depth': 10}
F1 0.596 params {'n_estimators': 350, 'max_depth': 4}


In [83]:
def objective(params):
    params = {
        'max_depth': int(params['max_depth']),
        'gamma': "{:.3f}".format(params['gamma']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'n_estimators': int(params['n_estimators'])
    }
    
    clf = xgb.XGBClassifier(
        n_jobs=2,
        **params
    )
    
    score = cross_val_score(clf, train2[traincols], train2.RESULT, scoring='f1', cv=KFold(5)).mean()
    print("F1 {:.3f} params {}".format(score, params))
    return -score

space = {
    'max_depth': hp.quniform('max_depth', 2, 8, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
    'gamma': hp.uniform('gamma', 0.0, 0.5),
    'n_estimators': hp.quniform('n_estimators', 25, 500, 25)
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)

F1 0.635 params {'max_depth': 4, 'gamma': '0.146', 'colsample_bytree': '0.336', 'n_estimators': 375}
F1 0.630 params {'max_depth': 3, 'gamma': '0.315', 'colsample_bytree': '0.722', 'n_estimators': 150}
F1 0.637 params {'max_depth': 4, 'gamma': '0.196', 'colsample_bytree': '0.383', 'n_estimators': 475}
F1 0.634 params {'max_depth': 4, 'gamma': '0.318', 'colsample_bytree': '0.631', 'n_estimators': 100}
F1 0.625 params {'max_depth': 8, 'gamma': '0.026', 'colsample_bytree': '0.327', 'n_estimators': 25}
F1 0.631 params {'max_depth': 8, 'gamma': '0.309', 'colsample_bytree': '0.985', 'n_estimators': 50}
F1 0.637 params {'max_depth': 7, 'gamma': '0.479', 'colsample_bytree': '0.472', 'n_estimators': 375}
F1 0.636 params {'max_depth': 6, 'gamma': '0.241', 'colsample_bytree': '0.818', 'n_estimators': 100}
F1 0.628 params {'max_depth': 3, 'gamma': '0.098', 'colsample_bytree': '0.327', 'n_estimators': 125}
F1 0.630 params {'max_depth': 3, 'gamma': '0.392', 'colsample_bytree': '0.559', 'n_estimators

In [84]:
def objective(params):
    params = {
        'max_depth': int(params['max_depth']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'n_estimators': int(params['n_estimators'])
    }
    
    clf = lgb.LGBMClassifier(
        **params
    )
    
    score = cross_val_score(clf, train2[traincols], train2.RESULT, scoring='f1', cv=KFold(5)).mean()
    print("F1 {:.3f} params {}".format(score, params))
    return -score

space = {
    'max_depth': hp.quniform('max_depth', 2, 8, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
    'n_estimators': hp.quniform('n_estimators', 25, 1000, 50)
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)

F1 0.634 params {'max_depth': 8, 'colsample_bytree': '0.460', 'n_estimators': 450}
F1 0.634 params {'max_depth': 7, 'colsample_bytree': '1.000', 'n_estimators': 400}
F1 0.636 params {'max_depth': 7, 'colsample_bytree': '0.851', 'n_estimators': 150}
F1 0.635 params {'max_depth': 6, 'colsample_bytree': '0.795', 'n_estimators': 200}
F1 0.632 params {'max_depth': 6, 'colsample_bytree': '0.413', 'n_estimators': 750}
F1 0.638 params {'max_depth': 4, 'colsample_bytree': '0.418', 'n_estimators': 250}
F1 0.635 params {'max_depth': 2, 'colsample_bytree': '0.787', 'n_estimators': 700}
F1 0.632 params {'max_depth': 8, 'colsample_bytree': '0.779', 'n_estimators': 400}
F1 0.630 params {'max_depth': 6, 'colsample_bytree': '0.686', 'n_estimators': 50}
F1 0.633 params {'max_depth': 7, 'colsample_bytree': '0.546', 'n_estimators': 650}


In [86]:
def objective(params):
    params = {'n_estimators': int(params['n_estimators']), 'max_depth': int(params['max_depth']),'max_features': float(params['max_features'])}
    clf = GradientBoostingClassifier(**params, random_state=123)
    score = cross_val_score(clf, train2[traincols], train2.RESULT, scoring='f1', cv=KFold(5)).mean()
    print("F1 {:.3f} params {}".format(score, params))
    return -score

space = {
    'n_estimators': hp.quniform('n_estimators', 25, 1000, 50),
    'max_depth': hp.quniform('max_depth', 1, 10, 1),
    'max_features': hp.quniform('max_features',.4,1,.1)
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)

KeyboardInterrupt: 

#### Ensemble ######

In [116]:
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def predict_proba(self, x):
        return self.clf.predict_proba(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)

In [114]:
SEED = 123 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(n_splits= NFOLDS, random_state=SEED)

def get_oof(clf, x_train, y_train, x_test):
    ntrain = x_train.shape[0]
    ntest = x_test.shape[0]
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict_proba(x_te)[:,1]
        oof_test_skf[i, :] = clf.predict_proba(x_test)[:,1]

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [90]:
rf_params = {'n_estimators': 325, 'max_depth': 10, 'random_state': 123}
gbm_params = {'n_estimators': 475, 'max_depth': 7, 'max_features': 0.9, 'random_state': 123}
xgb_params = {'max_depth': 7, 'gamma': '0.479', 'colsample_bytree': '0.472', 'n_estimators': 375, 'random_state':123}
lgb_params = {'max_depth': 4, 'colsample_bytree': '0.418', 'n_estimators': 250, 'random_state':123}
et_params = {'n_estimators': 500, 'max_depth': 10, 'random_state': 123}

In [118]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
gbm = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gbm_params)
xgb = SklearnHelper(clf=XGBClassifier, seed=SEED, params=xgb_params)
lgb = SklearnHelper(clf=LGBMClassifier, seed=SEED, params=lgb_params)

In [111]:
train2.shape

(20426, 714)

In [94]:
x_train = train2[traincols].values
y_train = train2.RESULT.values
x_test = test[traincols].values

In [119]:
rf_oof_train, rf_oof_test = get_oof(rf, x_train, y_train, x_test)
et_oof_train, et_oof_test = get_oof(et, x_train, y_train, x_test)
gbm_oof_train, gbm_oof_test = get_oof(gbm, x_train, y_train, x_test)
xgb_oof_train, xgb_oof_test = get_oof(xgb, x_train, y_train, x_test)
lgb_oof_train, lgb_oof_test = get_oof(lgb, x_train, y_train, x_test)

In [121]:
train_pred = pd.DataFrame()
train_pred['rf'] = rf_oof_train.reshape(1,-1).tolist()[0]
train_pred['et'] = et_oof_train.reshape(1,-1).tolist()[0]
train_pred['gbm'] = gbm_oof_train.reshape(1,-1).tolist()[0]
train_pred['xgb'] = xgb_oof_train.reshape(1,-1).tolist()[0]
train_pred['lgb'] = lgb_oof_train.reshape(1,-1).tolist()[0]
train_pred['actual'] = train2.RESULT.values
train_pred.to_csv('../data/blend_train.csv',index=False)

In [123]:
test_pred = pd.DataFrame()
test_pred['rf'] = rf_oof_test.reshape(1,-1).tolist()[0]
test_pred['et'] = et_oof_test.reshape(1,-1).tolist()[0]
test_pred['gbm'] = gbm_oof_test.reshape(1,-1).tolist()[0]
test_pred['xgb'] = xgb_oof_test.reshape(1,-1).tolist()[0]
test_pred['lgb'] = lgb_oof_test.reshape(1,-1).tolist()[0]
test_pred.to_csv('../data/blend_test.csv',index=False)

In [135]:
train_pred['ensemble'] = (np.round(train_pred.rf) + np.round(train_pred.et) + np.round(train_pred.gbm) + np.round(train_pred.xgb) + np.round(train_pred.lgb))*.2

In [136]:
for col in ['rf','et','gbm','xgb','lgb','ensemble']:
    print (col, f1_score(train_pred.actual,np.round(train_pred[col].values)))
    cutoffs = []
    f1s = []
    for cutoff in np.arange(.1,.5,.01):
        cutoffs.append(cutoff)
        f1s.append(f1_score(train_pred.actual,train_pred[col].values>cutoff))
    print (max(f1s),cutoffs[np.array(f1s).argmax()])    

rf 0.6130380502632579
0.6716632363791698 0.3599999999999999
et 0.5959234479539443
0.6670373665480428 0.3799999999999999
gbm 0.6301325747410229
0.6785920369301789 0.24999999999999992
xgb 0.6343669250645996
0.67867794255832 0.2799999999999999
lgb 0.6401792545633402
0.684940394128619 0.32999999999999985
ensemble 0.638620839206332
0.672023757533409 0.1


In [134]:
test_pred['ensemble'] = (test_pred.rf + test_pred.et + test_pred.gbm + test_pred.xgb + test_pred.lgb)*.2
test_pred['actual'] = np.round(test_pred.ensemble) #test_pred.ensemble > .35
print (test_pred.actual.value_counts(normalize=True))

0.0    0.662285
1.0    0.337715
Name: actual, dtype: float64
